In [7]:
import threading
import time
import os
from nbclient import NotebookClient
from nbformat import read

def executar_script(script_path):
    if os.path.isfile(script_path):
        try:
            print(f"Executando {script_path}...")

            # Abrir o notebook
            with open(script_path) as f:
                nb = read(f, as_version=4)

            # Executar o notebook
            client = NotebookClient(nb)
            client.execute()

            print(f"{script_path} executado com sucesso.")
        except Exception as e:
            print(f"Erro ao executar {script_path}: {e}")
    else:
        print(f"O arquivo {script_path} não foi encontrado.")

# Caminhos para os arquivos
path_extracao_onibus = 'extracaoOnibus.ipynb'
path_extracao_pontos = 'extracaoPontos.ipynb'
path_extracao_ocorrencia = 'extracaoOcorrencia.ipynb'

# Funções para executar scripts de extração em loops
def tarefa_extracao_onibus():
    while True:
        executar_script(path_extracao_onibus)
        time.sleep(60)  # Espera 1 minuto

def tarefa_extracao_pontos():
    while True:
        executar_script(path_extracao_pontos)
        time.sleep(10)  # Espera 10 segundos

def tarefa_extracao_ocorrencia():
    while True:
        executar_script(path_extracao_ocorrencia)
        time.sleep(900)  # Espera 15 minutos 

# Iniciando as threads
thread_onibus = threading.Thread(target=tarefa_extracao_onibus)
thread_pontos = threading.Thread(target=tarefa_extracao_pontos)
thread_ocorrencia = threading.Thread(target=tarefa_extracao_ocorrencia)

thread_onibus.start()
thread_pontos.start()
thread_ocorrencia.start()

# Aguarda a thread do ônibus terminar (que não deve, já que é uma execução contínua)
thread_onibus.join()

print("Todas as tarefas foram iniciadas.")


Executando extracaoPontos.ipynb...
Executando extracaoOcorrencia.ipynb...
extracaoOcorrencia.ipynb executado com sucesso.
extracaoPontos.ipynb executado com sucesso.
Executando extracaoPontos.ipynb...
Executando extracaoOcorrencia.ipynb...
extracaoOcorrencia.ipynb executado com sucesso.
extracaoPontos.ipynb executado com sucesso.
Executando extracaoPontos.ipynb...
extracaoPontos.ipynb executado com sucesso.
Executando extracaoPontos.ipynb...
Executando extracaoOcorrencia.ipynb...
extracaoOcorrencia.ipynb executado com sucesso.
extracaoPontos.ipynb executado com sucesso.
Executando extracaoPontos.ipynb...
extracaoPontos.ipynb executado com sucesso.
Executando extracaoPontos.ipynb...
Executando extracaoOcorrencia.ipynb...
extracaoOcorrencia.ipynb executado com sucesso.


KeyboardInterrupt: 

In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import threading
import time
import os

# Inicializar o app Dash
app = dash.Dash(__name__)

# Dados simulados (você pode substituí-los pela lógica real)
data = {
    "Script": ["extracaoOnibus.ipynb", "extracaoPontos.ipynb", "extracaoOcorrencia.ipynb"],
    "Status": ["Parado", "Parado", "Parado"],
    "Última Execução": [None, None, None],
}

# DataFrame para armazenar o status dos scripts
df_status = pd.DataFrame(data)

# Variável de controle para a execução dos scripts
executar = True

# Layout do dashboard
app.layout = html.Div([
    html.H1("Dashboard de Execução de Notebooks"),
    dcc.Interval(id='interval-component', interval=5*1000, n_intervals=0),  # Atualiza a cada 5 segundos
    dcc.Graph(id='live-update-graph'),
    html.Div(id='output-status'),
    html.Button("Parar Execução", id='stop-button', n_clicks=0)
])

# Callback para atualizar o gráfico e o status
@app.callback(
    Output('live-update-graph', 'figure'),
    Output('output-status', 'children'),
    Input('interval-component', 'n_intervals'),
    Input('stop-button', 'n_clicks')
)
def update_graph(n, stop_clicks):
    global executar
    if stop_clicks > 0:
        executar = False  # Parar a execução dos scripts
    
    # Atualizar o status dos scripts
    for index in range(len(df_status)):
        if df_status.at[index, "Status"] == "Executando" and not executar:
            df_status.at[index, "Status"] = "Parado"

    # Criar um gráfico simples para visualizar o status
    figure = {
        'data': [
            {
                'x': df_status["Script"],
                'y': df_status["Status"].apply(lambda x: 1 if x == "Concluído" else 0),
                'type': 'bar',
                'name': 'Status'
            }
        ],
        'layout': {
            'title': 'Status dos Scripts'
        }
    }

    # Exibir o status atualizado
    status_output = html.Div([
        html.H4("Status Atual:"),
        html.Table(
            # Cria a tabela com o cabeçalho e as linhas do DataFrame
            [html.Tr([html.Th(col) for col in df_status.columns])] +
            [html.Tr([html.Td(df_status.iloc[i][col]) for col in df_status.columns]) for i in range(len(df_status))]
        )
    ])

    return figure, status_output

# Função para executar os scripts (simplificada)
def executar_script(script_path, index):
    global executar
    while True:
        if os.path.isfile(script_path) and executar:
            df_status.at[index, "Status"] = "Executando"
            time.sleep(5)  # Simula a execução do script
            df_status.at[index, "Status"] = "Concluído"
            df_status.at[index, "Última Execução"] = time.strftime("%H:%M:%S")
        else:
            df_status.at[index, "Status"] = "Parado"
            break  # Parar a execução se a variável de controle for False
        time.sleep(60)

# Iniciando as threads para cada script
path_extracao_onibus = 'extracaoOnibus.ipynb'
path_extracao_pontos = 'extracaoPontos.ipynb'
path_extracao_ocorrencia = 'extracaoOcorrencia.ipynb'

thread_onibus = threading.Thread(target=executar_script, args=(path_extracao_onibus, 0), daemon=True)
thread_pontos = threading.Thread(target=executar_script, args=(path_extracao_pontos, 1), daemon=True)
thread_ocorrencia = threading.Thread(target=executar_script, args=(path_extracao_ocorrencia, 2), daemon=True)

thread_onibus.start()
thread_pontos.start()
thread_ocorrencia.start()

# Iniciar o servidor
if __name__ == '__main__':
    app.run_server(debug=True)


Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ipynb executado com sucesso.
Executando extracaoOnibus.ipynb...
extracaoOnibus.ip